In [29]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pythainlp
import pythaispell
from gingerit.gingerit import GingerIt
from langdetect import detect
import nltk
from keras import *
from keras.utils import to_categorical
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import dict_trie
from pythainlp.tag.named_entity import ThaiNameTagger

from sklearn.model_selection import train_test_split

In [30]:
# import data
df = pd.read_csv('../Description_classification/Product_description.csv')
df.describe()

,Category
count,61.000000
mean,6.475410
std,4.064507
min,1.000000
25%,3.000000
50%,6.000000
75%,10.000000
max,13.000000


In [31]:
df.head()

,Product,Description,Category
0,INOVO,INOVO I 812 V1 (Ram:3GB/Rom:32GB) รับประกัน 1 ...,1
1,Pinno,"ขนาดหน้าจอ 2.4 นิ้ว,ระบบ Dual SIM (2 ซิม),รองร...",1
2,R9,มือถือจอหยดน้ำหน้าจอใหญ่สุดและสเปคแท้คุ้มค่าที...,1
3,J5pro (OEM),สินค้าลดล้างสต๊อกประกัน 7 วันเท่านั้น ไม่รองรั...,1
4,J9 Plus,J9 Plusมือถือ OEM ตอบโจทย์ทางเลือกสำหรับงบจำกั...,1


In [32]:
x = df['Description'].values
y = df['Category'].values

In [33]:
y.shape

(61,)

In [34]:
x[0]

'INOVO I 812 V1 (Ram:3GB/Rom:32GB) รับประกัน 1 ปี ,จอ 6.0" QHD  Quad-Core Processor Ram 3 GB/Rom 32 GB แบต 2500 mAh Battery ,5.0 MP+8.0MP Pixel , Face uniock, Android 8'

In [35]:
# custom dictionary
path_custom_dict = "./custom_dict.txt"

with open(path_custom_dict, 'r',encoding="utf8") as f:
    new_words = f.read().splitlines()
    
words = set(new_words).union(thai_words())
custom_dictionary = dict_trie(words)

In [36]:
import re
import string

# word correction for EN
#parser = GingerIt()

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

# clean the text
def clean_msg(msg): 
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    msg = re.sub(r'✅','',msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

# split to words from text
def split_word(text):
    words = word_tokenize(text,engine='newmm',custom_dict=custom_dictionary,keep_whitespace=False)

    # Remove stop words TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]
    
    # รากศัพท์
    # EN
    words = [p_stemmer.stem(i) for i in words]
    
    # TH
    tokens_temp=[]
    for i in words:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            if(w_syn[0].lemma_names('tha')[0]=='แบต'):
                tokens_temp.append(w_syn[0].lemma_names('tha')[1])
            elif(w_syn[0].lemma_names('tha')[0].isdigit()):
                tokens_temp.append(w_syn[0].lemma_names('tha')[2])
            else:
                tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    words = tokens_temp
    
    # delete whitespace
    #words = [i for i in words if not ' ' in i]

    return words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [37]:
clean_text = [clean_msg(text) for text in x]
print(clean_text)

['INOVO I 812 V1 Ram3GBRom32GB รับประกัน 1 ปี จอ 60 QHD QuadCore Processor Ram 3 GBRom 32 GB แบต 2500 mAh Battery 50 MP80MP Pixel Face uniock Android 8', 'ขนาดหน้าจอ 24 นิ้วระบบ Dual SIM 2 ซิมรองรับ SD card สูงสุด 8GBปุ่มกดใหญ่สะใจ กดง่าย เห็นชัดช่องเสียบหูฟัง ขนาด 35 mmฟังค์ชั่นเครื่อง MP3 MP4 FM กล้อง บันทึกเสียง นาฬิกาปลุก ไฟฉายภาษาที่รองรับ English Russian French Spanish IndonesiaMelayu Thaiความถี่ Dual band GSM900 1800MHz', 'มือถือจอหยดน้ำหน้าจอใหญ่สุดและสเปคแท้คุ้มค่าที่สุดในตลาดตอนนี้หน้าจอแสดงผลกว้าง 626 นิ้วระบบปฏิบัติการ Android ของแท้ดีที่สุดสำหรับธุรกรรมการเงิน ปลอดภัย No rootพร้อม Google Play Protectช่วยป้องกันแอปที่เป็นอันตรายซอฟแวร์อัพเดทได้ ไม่ลอยแพฝาหลังแบบgeraldine color 2 เฉดสี ถอดฝาหลังได้รองรับ 2 ซิม ซิม3G 4G AIS TRUE DTAC MybyCATรองรับระบบรักษาความปลอดภัยด้วยการสแกนใบหน้ากล้องหลัง 8 MP พร้อมแฟลชกล้องหน้า 5 MP พร้อมโหมดหน้าสวยRAM 2GBROM 16 GB CPU MT65804Coreรองรับการสนทนาวีดิโอคอล Facebook live Lineแบตเตอรี่ 2800mAhรองรับการ์ดหน่วยความจำเสริมภายนอกแบบ microSD Card 

In [45]:
word_list = ['|'.join(split_word(txt)) for txt in clean_text]
print(word_list)

['inovo|I|812|วี|1|ram|3|gbrom|32|GB|รับประกัน|1|ปี|จอ|60|qhd|quadcor|processor|ram|3|gbrom|32|GB|แบตเตอรี่|2500|mah|batteri|50|MP|80|MP|pixel|face|uniock|android|8', 'ขนาดหน้าจอ|24|นิ้ว|ระบบ|dual|sim|2|ซิม|ค้ำ|SD|card|8|GB|ปุ่ม|สะใจ|กด|เห็นชัด|ช่อง|สอด|หูฟัง|ขนาด|35|mm|ฟัง|ค์ชั่น|เครื่อง|MP|3|MP|4|FM|กล้อง|อัดเทป|นาฬิกาปลุก|ไฟฉาย|ภาษา|ค้ำ|english|russian|french|spanish|indonesiamelayu|thai|ความถี่|dual|band|gsm|900|1800|mhz', 'มือถือ|จอ|หยดน้ำ|หน้าจอ|สเปค|คุ้มค่า|ตลาด|ตอนนี้|หน้า|จอแสดงผล|626|นิ้ว|ระบบปฏิบัติการ|android|ของแท้|ดี|สำหรับ|ธุรกรรม|การเงิน|ปลอดภัย|No|root|googl|play|protect|ป้องกัน|แอ|ป|ทำอันตราย|ซอ|ฟแวร์|อัพเดท|ลอยแพ|ฝา|geraldin|color|2|เฉดสี|ถอด|ฝา|ค้ำ|2|ซิม|ซิม|3|G|4|G|ai|true|dtac|mybycat|ค้ำ|ระบบ|รักษา|ความปลอดภัย|สแกน|หน้า|กล้อง|8|MP|แฟลช|กล้องหน้า|5|MP|โหมด|หน้า|สวย|ram|2|gbrom|16|GB|cpu|MT|65804|core|ค้ำ|คุย|วีดิโอคอล|facebook|live|line|แบตเตอรี่|2800|mah|ค้ำ|การ์ด|หน่วยความจำเสริม|microsd|card|ขนาด|64|GB|ตัว|เครื่อง|ภาษาไทย|สินค้า|รับประกัน|1|ปี|0|บริการ|ไท|เคลม|

In [57]:
from keras.preprocessing import text
from keras.preprocessing import sequence

max_word = 5000
maxlen = 60

# Define Tokenizer
tokenizer = text.Tokenizer(num_words=max_word,split='|') #define corpus  to keep maximum number of words 
tokenizer.fit_on_texts(word_list)
tokenizer.word_index

{'1': 1,
 'การรับประกัน': 2,
 'x': 3,
 'สินค้า': 4,
 'ชนิด': 5,
 '2': 6,
 'ผ้า': 7,
 '3': 8,
 'กล้อง': 9,
 'gb': 10,
 'รุ่น': 11,
 '4': 12,
 'สี': 13,
 'นิ้ว': 14,
 '5': 15,
 'กล่อง': 16,
 'ขนาด': 17,
 'ภาพ': 18,
 'แบรนด์': 19,
 'แบตเตอรี่': 20,
 'hd': 21,
 'ตัว': 22,
 'th': 23,
 'usb': 24,
 'ละเอียด': 25,
 'ค้ำ': 26,
 'น': 27,
 'วัสดุ': 28,
 'ช่อง': 29,
 'ระยะเวลา': 30,
 'ทำ': 31,
 'ปี': 32,
 'สำหรับ': 33,
 'ข้อมูล': 34,
 'mp': 35,
 'พิกเซล': 36,
 'เอ': 37,
 'วิดีโอ': 38,
 'ระบบ': 39,
 'ดี': 40,
 'core': 41,
 'เลนส์': 42,
 '8': 43,
 'ใช้': 44,
 'ไม่มี': 45,
 'หนา': 46,
 'หน้าจอ': 47,
 '30': 48,
 'pc': 49,
 'หนึ่งร้อย': 50,
 'แสง': 51,
 'ชิ้น': 52,
 'no': 53,
 'ชาร์จ': 54,
 'สาย': 55,
 'intel': 56,
 'ram': 57,
 'เครื่อง': 58,
 'โหมด': 59,
 '7': 60,
 'ประกัน': 61,
 'ล้าน': 62,
 'คุณภาพ': 63,
 '6': 64,
 'เดือน': 65,
 'wifi': 66,
 'ไฟ': 67,
 '•': 68,
 'ขนาดหน้าจอ': 69,
 'หน้า': 70,
 'ชอบ': 71,
 'ประมวลผล': 72,
 'คุณสมบัติ': 73,
 'ความร้อน': 74,
 'รองเท้าแตะ': 75,
 'ผ้าปูที่นอน': 76,
 'รับ

In [58]:
tokenizer.num_words

5000

In [59]:
# Preprocess data function
def preprocess_fn(data):
    sequeces = tokenizer.texts_to_sequences(data)
    padding_sequences = sequence.pad_sequences(sequeces,maxlen = maxlen)
    
    return padding_sequences,sequeces

In [60]:
x_preprocess,x_seq = preprocess_fn(word_list)
print(x_seq)

[[864, 240, 865, 98, 1, 57, 8, 393, 122, 10, 77, 1, 32, 162, 291, 543, 292, 394, 57, 8, 393, 122, 10, 20, 395, 99, 163, 241, 35, 196, 35, 866, 544, 867, 164, 43], [69, 100, 14, 39, 396, 868, 6, 293, 26, 397, 242, 43, 10, 243, 869, 545, 870, 29, 101, 294, 17, 244, 123, 871, 872, 58, 35, 8, 35, 12, 546, 9, 873, 874, 875, 124, 26, 876, 877, 878, 879, 880, 881, 295, 396, 882, 883, 884, 547, 296], [125, 162, 548, 47, 885, 886, 549, 887, 70, 126, 888, 14, 245, 164, 246, 40, 33, 889, 890, 297, 53, 891, 892, 550, 893, 165, 551, 141, 894, 895, 896, 197, 897, 166, 898, 899, 6, 900, 298, 166, 26, 6, 293, 293, 8, 102, 12, 102, 901, 902, 903, 904, 26, 39, 905, 906, 907, 70, 9, 43, 35, 142, 247, 15, 35, 59, 70, 103, 57, 6, 393, 86, 10, 127, 552, 908, 41, 26, 909, 910, 911, 553, 398, 20, 554, 99, 26, 78, 912, 913, 242, 17, 167, 10, 22, 58, 555, 4, 77, 1, 32, 556, 399, 299, 914, 915, 60, 916, 4, 917, 918, 919, 27, 920, 921, 44], [4, 300, 922, 61, 60, 26, 923, 924, 557, 925, 26, 301, 558, 559, 560, 292

In [61]:
print(x_preprocess)

[[   0    0    0 ...  867  164   43]
 [   0    0    0 ...  884  547  296]
 [ 907   70    9 ...  920  921   44]
 ...
 [ 389 1902    6 ...  528  229  231]
 [  27  187 1920 ...  845  846  527]
 [   0    0    0 ...  392  510  863]]


In [39]:
from pythainlp.tag import pos_tag_sents

pos_pud = pos_tag_sents(word_list, corpus='pud')

In [40]:
pos_tag_sents(word_list, corpus='orchid_ud')

[[('inovo', 'NOUN'),
  ('I', 'NOUN'),
  ('812', 'NUM'),
  ('วี', 'NOUN'),
  ('1', 'NUM'),
  ('ram', 'NOUN'),
  ('3', 'NUM'),
  ('gbrom', 'NOUN'),
  ('32', 'NUM'),
  ('GB', 'NOUN'),
  ('รับประกัน', 'NOUN'),
  ('1', 'NUM'),
  ('ปี', 'NOUN'),
  ('จอ', 'NOUN'),
  ('60', 'NUM'),
  ('qhd', 'NOUN'),
  ('quadcor', 'NOUN'),
  ('processor', 'NOUN'),
  ('ram', 'NOUN'),
  ('3', 'NUM'),
  ('gbrom', 'NOUN'),
  ('32', 'NUM'),
  ('GB', 'NOUN'),
  ('แบตเตอรี่', 'NOUN'),
  ('2500', 'NUM'),
  ('mah', 'NOUN'),
  ('batteri', 'NOUN'),
  ('50', 'NUM'),
  ('MP', 'NOUN'),
  ('80', 'NUM'),
  ('MP', 'NOUN'),
  ('pixel', 'NOUN'),
  ('face', 'NOUN'),
  ('uniock', 'NOUN'),
  ('android', 'NOUN'),
  ('8', 'NUM')],
 [('ขนาดหน้าจอ', 'NOUN'),
  ('24', 'NUM'),
  ('นิ้ว', 'NOUN'),
  ('ระบบ', 'NOUN'),
  ('dual', 'NOUN'),
  ('sim', 'NOUN'),
  ('2', 'NUM'),
  ('ซิม', 'NOUN'),
  ('ค้ำ', 'NOUN'),
  ('SD', 'NOUN'),
  ('card', 'NOUN'),
  ('8', 'NUM'),
  ('GB', 'NOUN'),
  ('ปุ่ม', 'NOUN'),
  ('สะใจ', 'NOUN'),
  ('กด', 'VERB'),
  

In [41]:
pos_tag_sents(word_list)

[[('inovo', 'NCMN'),
  ('I', 'NCMN'),
  ('812', 'NCNM'),
  ('วี', 'CMTR'),
  ('1', 'NCNM'),
  ('ram', 'NCMN'),
  ('3', 'NCNM'),
  ('gbrom', 'NCMN'),
  ('32', 'NCNM'),
  ('GB', 'NCMN'),
  ('รับประกัน', 'CNIT'),
  ('1', 'DCNM'),
  ('ปี', 'CMTR'),
  ('จอ', 'CNIT'),
  ('60', 'DCNM'),
  ('qhd', 'NCMN'),
  ('quadcor', 'NCMN'),
  ('processor', 'NCMN'),
  ('ram', 'NCMN'),
  ('3', 'NCNM'),
  ('gbrom', 'NCMN'),
  ('32', 'NCNM'),
  ('GB', 'NCMN'),
  ('แบตเตอรี่', 'NCMN'),
  ('2500', 'NCNM'),
  ('mah', 'NCMN'),
  ('batteri', 'CMTR'),
  ('50', 'DCNM'),
  ('MP', 'CMTR'),
  ('80', 'DCNM'),
  ('MP', 'CMTR'),
  ('pixel', 'NCMN'),
  ('face', 'NCMN'),
  ('uniock', 'NCMN'),
  ('android', 'NCMN'),
  ('8', 'NCNM')],
 [('ขนาดหน้าจอ', 'NCMN'),
  ('24', 'DCNM'),
  ('นิ้ว', 'CMTR'),
  ('ระบบ', 'NCMN'),
  ('dual', 'NCMN'),
  ('sim', 'NCMN'),
  ('2', 'NCNM'),
  ('ซิม', 'CMTR'),
  ('ค้ำ', 'CNIT'),
  ('SD', 'NCMN'),
  ('card', 'NCMN'),
  ('8', 'DCNM'),
  ('GB', 'NCMN'),
  ('ปุ่ม', 'CNIT'),
  ('สะใจ', 'NCMN'),
  ('ก

In [96]:
filtered = []
for idx,value in enumerate(pos_pud):
    tmp = []
    for p0,p1 in value:
        tmpp = []
        if(p1=='NUM') or (p1=='NOUN'):
            tmpp.append(p0)
            tmpp.append(p1)
            tmp.append(tmpp)
    print(tmp)
    
           
        

[['I', 'NOUN'], ['812', 'NUM'], ['วี', 'NOUN'], ['1', 'NUM'], ['ram', 'NOUN'], ['3', 'NUM'], ['gbrom', 'NOUN'], ['32', 'NUM'], ['GB', 'NOUN'], ['1', 'NUM'], ['ปี', 'NOUN'], ['จอ', 'NOUN'], ['60', 'NUM'], ['qhd', 'NOUN'], ['quadcor', 'NOUN'], ['processor', 'NOUN'], ['ram', 'NOUN'], ['3', 'NUM'], ['gbrom', 'NOUN'], ['32', 'NUM'], ['GB', 'NOUN'], ['แบตเตอรี่', 'NOUN'], ['2500', 'NUM'], ['batteri', 'NOUN'], ['50', 'NUM'], ['MP', 'NOUN'], ['80', 'NUM'], ['MP', 'NOUN'], ['pixel', 'NOUN'], ['face', 'NOUN'], ['uniock', 'NOUN'], ['android', 'NOUN'], ['8', 'NUM']]
[['ขนาดหน้าจอ', 'NOUN'], ['24', 'NUM'], ['นิ้ว', 'NOUN'], ['ระบบ', 'NOUN'], ['dual', 'NOUN'], ['sim', 'NOUN'], ['2', 'NUM'], ['ซิม', 'NOUN'], ['SD', 'NOUN'], ['card', 'NOUN'], ['8', 'NUM'], ['GB', 'NOUN'], ['สะใจ', 'NOUN'], ['ช่อง', 'NOUN'], ['สอด', 'NOUN'], ['หูฟัง', 'NOUN'], ['ขนาด', 'NOUN'], ['35', 'NUM'], ['mm', 'NOUN'], ['เครื่อง', 'NOUN'], ['MP', 'NOUN'], ['3', 'NUM'], ['MP', 'NOUN'], ['4', 'NUM'], ['FM', 'NOUN'], ['อัดเทป', 'NOU

# Create model

In [64]:
tokens_list_j = [','.join(tkn) for tkn in word_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [30]:
cvec.vocabulary_

{'inovo': 530,
 'I': 285,
 '812': 223,
 'V': 315,
 '1': 11,
 'ram': 687,
 '3': 106,
 'gbrom': 478,
 '32': 114,
 'GB': 279,
 'รับประกัน': 1411,
 'ปี': 1263,
 'จอ': 998,
 '60': 194,
 'qhd': 682,
 'quadcor': 684,
 'processor': 671,
 'แบตเตอรี่': 1844,
 '2500': 90,
 'mah': 578,
 'batteri': 356,
 '50': 161,
 'MP': 293,
 '80': 219,
 'pixel': 659,
 'face': 448,
 'uniock': 783,
 'android': 337,
 '8': 218,
 'ขนาดหน้าจอ': 905,
 '24': 81,
 'นิ้ว': 1198,
 'ระบบ': 1399,
 'dual': 428,
 'sim': 725,
 '2': 64,
 'ซิม': 1055,
 'ค้ำ': 991,
 'SD': 306,
 'card': 378,
 'ปุ่มกด': 1265,
 'สะใจ': 1524,
 'กด': 836,
 'เห็นชัด': 1807,
 'ช่อง': 1041,
 'สอด': 1515,
 'หูฟัง': 1613,
 'ขนาด': 904,
 '35': 124,
 'mm': 595,
 'ฟัง': 1326,
 'ค์ชั่น': 992,
 'เครื่อง': 1690,
 '4': 135,
 'FM': 277,
 'กล้อง': 861,
 'อัดเทป': 1643,
 'นาฬิกาปลุก': 1193,
 'ไฟฉาย': 1937,
 'ภาษา': 1339,
 'english': 439,
 'russian': 702,
 'french': 469,
 'spanish': 740,
 'indonesiamelayu': 527,
 'thai': 767,
 'ความถี่': 948,
 'band': 355,
 'gsm': 498

In [31]:
len(cvec.vocabulary_)

1971

In [34]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','))
t_feat = tvec.fit_transform(tokens_list_j)

In [35]:
len(tvec.idf_)

1971

In [83]:
# Define model
lstm = models.Sequential()
lstm.add(layers.Embedding(max_word,512))
lstm.add(layers.LSTM(512))
lstm.add(layers.LeakyReLU(alpha=0.2))
#lstm.add(layers.Dense(128,activation='relu'))
lstm.add(layers.Dense(14,activation='softmax'))

# Compile model
lstm.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])
lstm.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 512)         2560000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 14)                7182      
Total params: 4,666,382
Trainable params: 4,666,382
Non-trainable params: 0
_________________________________________________________________


In [84]:
# Fit model
his =  lstm.fit(x_preprocess,y,epochs=20,batch_size=32)

C:\Users\super\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
61/61 [==============================] - 0s 8ms/step - loss: 2.6361 - acc: 0.1639
Epoch 2/20
61/61 [==============================] - 0s 2ms/step - loss: 2.5816 - acc: 0.7869
Epoch 3/20
61/61 [==============================] - 0s 2ms/step - loss: 2.5120 - acc: 0.8852
Epoch 4/20
61/61 [==============================] - 0s 2ms/step - loss: 2.3554 - acc: 0.4262
Epoch 5/20
61/61 [==============================] - 0s 2ms/step - loss: 2.5402 - acc: 0.3443
Epoch 6/20
61/61 [==============================] - 0s 2ms/step - loss: 2.1007 - acc: 0.6066
Epoch 7/20
61/61 [==============================] - 0s 2ms/step - loss: 2.0153 - acc: 0.4590
Epoch 8/20
61/61 [==============================] - 0s 2ms/step - loss: 1.8958 - acc: 0.4262
Epoch 9/20
61/61 [==============================] - 0s 2ms/step - loss: 1.7083 - acc: 0.4426
Epoch 10/20
61/61 [==============================] - 0s 2ms/step - loss: 1.5248 - acc: 0.4426
Epoch 11/20
61/61 [==============================] - 0s 2ms/step - lo

In [85]:
# Define an evaluation function to print the evaluation result
def evaluation_report(model,features,labels):
    
    # Calculate result
    result = model.evaluate(features,labels,verbose=False)
    
    # Predict and convert into a class
    pred_class = model.predict(features).argmax(axis=1)
    
    # Show report
    print(confusion_matrix(labels,pred_class))
    print(classification_report(labels,pred_class))
    print("Loss: %s Accuracy: %s" %(result[0],result[1]))
    
    return pred_class

In [86]:
evaluation_report(lstm,x_preprocess,y)

[[9 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 5 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 5 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 2 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 5 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 5]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         5
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         3
           6       0.83      1.00      0.91         5
           7       1.00      0.67      0.80         3
           8       1.00      1.00      1.00         5
           9       1.00      1.00      1.00         4
          10       1.00      0.33      0.50         3
          11       0.67    

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,
        3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,
        7,  8,  8,  8,  8,  8,  9,  9,  9,  9, 11, 10, 11, 11, 11, 11, 11,
       12, 12, 12, 12, 12, 13, 13, 13, 13, 13], dtype=int64)

In [92]:
text_test = ["หน้าจอ 6.4 นิ้ว ความละเอียด 3040x1440 Quad HD+ (550 ppi)กล้องหลัง 3 ตัว (16 MP f/2.2 (Ultra Wide),12 MP f/1.5 & 2.4 (Wide), 12 MP f/2.4 (Tele))กล้องหน้า 2 ตัว (10 MP f/1.9 (Main), 8 MP f/2.2 (Live Focus)หน่วยประมวลผล Octa-core Snapdragon 855, Octa-core Exynos9 9820Ram 8/512 GBแบตเตอรี่ 4,100 mAh (Typical)"]
clean_text_test = [clean_msg(text) for text in text_test]
word_list_test = ['|'.join(split_word(txt)) for txt in clean_text_test]
x_preprocess_test,x_seq_test = preprocess_fn(word_list_test)

pred = lstm.predict(x_preprocess_test).argmax(axis=1)
print(pred)

[1]
